<a href="https://colab.research.google.com/github/Ranjankraj/Shala/blob/main/multiclass_20_ng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Classification of 20_newsgroup consisting of around 20000 files with 20 classes with Naive bayes and LSTM

#Mounting drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip  "/content/20_newsgroups.zip"

In [ ]:
#After manually uploading zip file
!unzip -u "/content/20_newsgroups.zip" -d "/content/Internship/"

In [3]:
!ls

20_newsgroups.zip  Internship  sample_data


In [4]:
from os import listdir
from os.path import isfile, join
# import string

In [5]:
my_path = "/content/Internship/20_newsgroups"

#creating a list of folder names to make valid pathnames later
folders = [f for f in listdir(my_path)]

In [6]:
folders

['talk.politics.guns',
 'talk.politics.misc',
 'comp.graphics',
 'sci.electronics',
 'talk.politics.mideast',
 'sci.crypt',
 'rec.autos',
 'alt.atheism',
 'soc.religion.christian',
 'talk.religion.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'rec.sport.baseball',
 'sci.med',
 'sci.space',
 'rec.motorcycles',
 'misc.forsale',
 'comp.os.ms-windows.misc',
 'comp.windows.x',
 'rec.sport.hockey']

In [7]:
print(len(folders))

20


In [8]:
#function to remove incompatible files
import os
import sklearn.datasets
def find_incompatible_files(path):
    """
    Finds the filenames that are incompatible with `CountVectorizer`. These files are usually not compatible with UTF8!
    parameter `path` is the absolute or relative path of the training data's root directory.
    """

    count_vector = sklearn.feature_extraction.text.CountVectorizer()
    files = sklearn.datasets.load_files(path)
    num = []
    for i in range(len(files.filenames)):
        try:
            count_vector.fit_transform(files.data[i:i + 1])
        except UnicodeDecodeError:
            num.append(files.filenames[i])
            os.remove(files.filenames[i])
        except ValueError:
            pass

    return num


In [9]:
#Removing incompatible files
val = find_incompatible_files(my_path)

In [10]:
#printing no. of incompatible files removed

print(len(val))

73


In [11]:
#creating a 2D list to store list of all files in different folders

files = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    files.append([f for f in listdir(folder_path)])

In [12]:
#There is around 20000 files or data points to read from
sum(len(files[i]) for i in range(20))

19924

In [13]:
pathname_list = []
for fo in range(len(folders)):
    for fi in files[fo]:
        pathname_list.append(join(my_path, join(folders[fo], fi)))

In [14]:
len(pathname_list)

19924

In [15]:
#making an array containing the classes each of the documents belong to

Y = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    num_of_files= len(listdir(folder_path))
    for i in range(num_of_files):
        Y.append(folder_name)

In [16]:
len(Y)

19924

In [17]:
# function for reading data

def ps(doc):
      f = open(doc)
      data= f.read()
      return data

In [18]:
#Reading & storing data from all the files 

import glob
import os.path
import string 

X_data = []

for i in range(len(pathname_list)):
  for file in glob.glob(pathname_list[i]):
      
      data = ps(file)
      X_data.append(data)

In [19]:
print(len(X_data))

19924


In [20]:
#Sample of texts in one file 

print(X_data[1234])

Xref: cantaloupe.srv.cs.cmu.edu talk.politics.misc:178544 alt.sex:111457
Newsgroups: talk.politics.misc,ca.politics,ba.politics,alt.sex
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!uunet!pipex!ibmpcug!exnet!exnet2!sys1
From: sys1@exnet.co.uk (Xavier Gallagher)
Subject: Re: Using California's Antidiscrimination: The Sort Of Case I Predicted
Message-ID: <C5Jv9A.F1B@exnet.co.uk>
Organization: ExNet Systems Ltd Public Access News, London, UK
References: <15266@optilink.COM> <1993Apr08.092954.13507@armory.com> <15312@optilink.COM>
Date: Fri, 16 Apr 1993 00:12:44 GMT
Lines: 26

In article <15312@optilink.COM> cramer@optilink.COM (Clayton Cramer) writes:
>In article <1993Apr08.092954.13507@armory.com>, rstevew@armory.com (Richard Steven Walz) writes:
># Face it, Clayton, he was not found guilty, and so what if gays sometimes
># make it consensually with 16 year old boys. There ARE 16 year old gays, you
># know. And as I recall, the case of the state rested on the testimon

In [21]:
#Organising data in pandas dataframe to work with

import pandas as pd

df1 = pd.DataFrame(X_data, columns=['Text'])
df2 = pd.DataFrame(Y, columns=['Labels'])

df = pd.concat([df1, df2], axis=1)
print(len(df))
df.head()

19924


,Text,Labels
0,Newsgroups: talk.politics.guns\nPath: cantalou...,talk.politics.guns
1,Newsgroups: talk.politics.guns\nPath: cantalou...,talk.politics.guns
2,Xref: cantaloupe.srv.cs.cmu.edu talk.politics....,talk.politics.guns
3,Xref: cantaloupe.srv.cs.cmu.edu talk.politics....,talk.politics.guns
4,Newsgroups: talk.politics.guns\nPath: cantalou...,talk.politics.guns


In [22]:
# Cleaning the texts
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus = []

for i in range(len(X_data)):
    review = re.sub('[^a-zA-Z]', ' ', df['Text'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english')) and len(word)>3]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [30]:
df.Text = corpus

In [24]:
df.Labels.value_counts()

comp.os.ms-windows.misc     1000
talk.politics.mideast       1000
talk.politics.guns          1000
rec.motorcycles             1000
talk.politics.misc          1000
sci.crypt                    999
sci.med                      997
talk.religion.misc           997
sci.space                    997
soc.religion.christian       997
misc.forsale                 996
comp.windows.x               996
rec.autos                    996
comp.sys.ibm.pc.hardware     995
comp.graphics                995
sci.electronics              994
rec.sport.hockey             994
alt.atheism                  993
rec.sport.baseball           993
comp.sys.mac.hardware        985
Name: Labels, dtype: int64

In [31]:
df.Labels = df.Labels.astype('category')

df.dtypes

Text        object
Labels    category
Code          int8
dtype: object

In [32]:
df['Code'] = df.Labels.cat.codes
df.head()

,Text,Labels,Code
0,newsgroup talk polit gun path cantaloup news h...,talk.politics.guns,16
1,newsgroup talk polit gun path cantaloup crabap...,talk.politics.guns,16
2,xref cantaloup talk polit gun rush limbaugh ne...,talk.politics.guns,16
3,xref cantaloup talk polit gun cultur rush limb...,talk.politics.guns,16
4,newsgroup talk polit gun path cantaloup magnes...,talk.politics.guns,16


In [33]:
from sklearn.model_selection import train_test_split

X = df.Text
y = df.Code
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state = 123, shuffle=True)

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [35]:
def accuracy_summary(pipeline, X_train, y_train, X_test, y_test):
    get_fit = pipeline.fit(X_train, y_train)
    y_pred1 = get_fit.predict(X_test)
    accuracy1 = accuracy_score(y_test, y_pred)
    y_pred2 = get_fit.predict(X_train)
    accuracy2 = accuracy_score(y_train, y_pred)
    print("accuracy score for test: {0:.2f}%".format(accuracy1*100))
    print("accuracy score for train: {0:.2f}%".format(accuracy2*100))
    return accuracy

In [39]:
import numpy as np
cv = CountVectorizer()
rf = RandomForestClassifier()
n_features = np.arange(10000,25001,5000)

def nfeature_accuracy_checker(vectorizer=cv, n_features=n_features, ngram_range=(1, 1), classifier=rf):
    result = []
    print(classifier)
    print("\n")
    for n in n_features:
        vectorizer.set_params(max_features=n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print("Test result for {} features".format(n))
        nfeature_accuracy = accuracy_summary(checker_pipeline, X_train, y_train, X_test, y_test)
        result.append((n,nfeature_accuracy))
    return result

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [41]:
print("Result for trigram by random forrest (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(vectorizer=tfidf,ngram_range=(1, 3))

Result for trigram with stop words (Tfidf)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


Test result for 10000 features
accuracy score: 93.35%
Test result for 15000 features
accuracy score: 93.72%
Test result for 20000 features
accuracy score: 93.54%
Test result for 25000 features
accuracy score: 93.64%


In [ ]:
#Storing list of all the words
import pandas

list_of_all_words = []

for i in range(len(pathname_list)):
  for file in glob.glob(pathname_list[i]):
      
      words = ps(file)
      for word in words:
        list_of_all_words.append(word)


In [ ]:
import numpy as np

np_list_of_all_words = np.asarray(list_of_all_words)

words, counts = np.unique(np_list_of_all_words, return_counts=True)
len(words)

95960

In [ ]:
def count(word):
  d_fq = []
  for i in range(len(pathname_list)):
    for file in glob.glob(pathname_list[i]):

        f = open(file)
        data= f.read()
        words = cleanDoc(data)
        if word in words:
          d_fq.append(pathname_list[i])
  return len(d_fq)

In [ ]:
#dictionary will contain unique words mapped with their document frequency
d_words = {}

for word in words: d_words[word] = count(word)

In [ ]:
print(d_words)

In [ ]:
#Sorting dictionary
import operator

sorted_d = dict( sorted(d_words.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
#choosing 500 to 1000 words by document frequency
dict_items = sorted_d.keys()

f_t = list(dict_items)[500:1001]

print(f_t)

In [ ]:
#We can loop over the below function to find Probability(word|class)

def prob(word,class):
  d_fq = []
  
  for folder_name in folders:
    if class == folder_name:
      files = []
      folder_path = join(my_path, folder_name)
      files.append([f for f in listdir(folder_path)])
      pathname_list = []
      for fi in files[fo]:
          pathname_list.append(join(my_path, join(folders[fo], fi)))
      for i in range(len(pathname_list)):
        for file in glob.glob(pathname_list[i]):

          f = open(file)
          data= f.read()
          words = cleanDoc(data)
          if word in words:
            d_fq.append(pathname_list[i])

          return ((len(d_fq))/(len(pathname_list)))


w = 'article'
c = 'alt.atheism'

prob(w,c)

In [ ]:
#fitting the model 

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
print("Result for trigram by naive bayes (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(vectorizer=tfidf,ngram_range=(1, 3), classifier=clf)

In [ ]:
clf.fit(X_train, Y_train)

In [ ]:
Y_predict = clf.predict(X_test)

In [ ]:
#accuracy for test data
clf.score(X_test, Y_test)

0.7751455531017868

In [ ]:
#printing report on test
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(Y_test, Y_predict))

                          precision    recall  f1-score   support

             alt.atheism       0.74      0.76      0.75       251
           comp.graphics       0.66      0.66      0.66       267
 comp.os.ms-windows.misc       0.65      0.64      0.65       256
comp.sys.ibm.pc.hardware       0.60      0.65      0.63       233
   comp.sys.mac.hardware       0.71      0.71      0.71       267
          comp.windows.x       0.79      0.70      0.74       245
            misc.forsale       0.71      0.68      0.69       249
               rec.autos       0.73      0.82      0.77       236
         rec.motorcycles       0.81      0.88      0.84       234
      rec.sport.baseball       0.87      0.85      0.86       250
        rec.sport.hockey       0.84      0.96      0.90       228
               sci.crypt       0.88      0.92      0.90       239
         sci.electronics       0.73      0.69      0.71       272
                 sci.med       0.94      0.85      0.89       246
         

In [ ]:
Y_predict_tr = clf.predict(X_train)

In [ ]:
#accuracy for training data

clf.score(X_train, Y_train)

0.8584621561935354

In [ ]:
#printing report on training set
print(classification_report(Y_train, Y_predict_tr))

                          precision    recall  f1-score   support

             alt.atheism       0.80      0.87      0.83       742
           comp.graphics       0.80      0.78      0.79       728
 comp.os.ms-windows.misc       0.80      0.83      0.81       744
comp.sys.ibm.pc.hardware       0.78      0.78      0.78       762
   comp.sys.mac.hardware       0.79      0.83      0.81       718
          comp.windows.x       0.89      0.83      0.86       751
            misc.forsale       0.82      0.82      0.82       747
               rec.autos       0.86      0.89      0.87       760
         rec.motorcycles       0.88      0.93      0.91       766
      rec.sport.baseball       0.91      0.93      0.92       743
        rec.sport.hockey       0.91      0.96      0.94       766
               sci.crypt       0.92      0.92      0.92       760
         sci.electronics       0.85      0.84      0.85       722
                 sci.med       0.95      0.87      0.90       751
         

In [ ]:
#Building multiclass classification LSTM model

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences

In [ ]:
n_most_common_words = 4000
epochs = 10
emb_dim = 128
batch_size = 500
max_len = 200

X_train = pad_sequences(X_train, padding='post', maxlen=max_len)
X_test = pad_sequences(X_test, padding='post', maxlen=max_len)

In [ ]:
#Fitting layers for LSTM 
model = Sequential()
model.add(Embedding(n_most_common_words, emb_dim, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(128, dropout=0.4, recurrent_dropout=0.4)))
model.add(Dense(20, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
clsi = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, callback=[EarlyStopping(monitor='val_loss', mode='auto')])


In [ ]:
#metrics of the test set is calculated
accr_1 = model.evaluate(X_test, Y_test)

In [ ]:
#metrics of the train set is calculated
accr_2 = model.evaluate(X_train, Y_train)